In [1]:
import pandas as pd
import transformers
import numpy as np
import os
import json
import ast
# from samutil import SpacyTransformer
import spacy
model = spacy.load('en_core_web_sm')
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.feature_extraction.text import CountVectorizer

import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification

from sklearn.utils.extmath import softmax
import numpy as np



In [22]:
df = pd.read_json('../data/main/tmdb_data_main.json')
df = df[['tmdb_id','movie','overview','genres','genres_new','popularity','release_year','vote_average','budget','revenue','poster_path']]
df = df.dropna(subset=['genres'])

# Get Main Generes
---

In [23]:
genre_include = df.explode('genres').genres.value_counts().keys()[0:12]

In [24]:
def genres_check(film_genres):
    
    for genre in film_genres:
        if genre not in genre_include:
            return False
    return True

In [25]:
df['genres_valid'] = df['genres'].apply(genres_check)

In [27]:
df = df[df.genres_valid ==True]

In [28]:
df.shape

(15001, 12)